In [1]:
import os
import csv
import numpy as np
from ipwhois import IPWhois

In [2]:
data_dir = "./../Raw Data/"
destinations = [i[ : -4] for i in os.listdir(data_dir)]
n_destinations = len(destinations)
n_sources = 1

data = []

In [3]:
destinations

['schum',
 'virgin',
 'cambridge',
 'watch_jp',
 'talktalk',
 'tripmall',
 'cmu',
 'discord',
 'craig',
 'columbia']

In [4]:
def read_file(filename):
    output = []
    with open(filename) as f:
        data = f.readlines()
        output = [x.strip() for x in data]
    return output

In [5]:
IP_address_file = "./../Processed Data/IP_Address_list.txt"
AS_number_file = "./../Processed Data/AS_Number_list.txt"

ip_address_list = read_file(IP_address_file)
as_number_list = read_file(AS_number_file)

In [6]:
def get_AS_number(ip_address):
    if ip_address in ip_address_list:
        return as_number_list[ip_address_list.index(ip_address)]
    else:
        ip_address_list.append(ip_address)
        try:
            as_number = IPWhois(ip_address).lookup_rdap()['asn']
            if as_number == 'NA':
                as_number = -100
            elif as_number == '':
                as_number = -100
        except:
            as_number = -100
        as_number_list.append(as_number)
    return as_number

In [7]:
sources_label = [-1, -2, -3, -4, -5]
destinations_label = [-6, -7, -8, -9, -10, -11, -12, -13, -14, -15]
fields = ['Source - AS', 'Source - IP', 'Target - AS', 'Target - IP', 'RTTD']
edge_list = []

In [12]:
%%capture cap --no-stderr
tmp = 1
flag = 0
tmp_edge_list = []

for i_dest, destination in enumerate(destinations):
    source_i = -1
    with open(data_dir + destination + ".txt", "r") as f:
        lines = f.readlines()
        lines = [line.strip().split() for line in lines]
        prev_as_number, prev_ip_address, rttd_prev = -100, 0, 0 
        
        for i, line in enumerate(lines):
            if 'traceroute' in line or 'Tracing' in line or 'Trace' in line:
                source_i += 1
                prev_as_number, rttd_prev = -100, 0
                flag = 1
                print(f"\n")
                continue
            
            print(line)
            
            if line == []:
                continue
            elif '*' in line:
                continue
            
            if source_i == 0:
                if line[2] == 'ms':
                    rttd = np.average([int(line[1]), int(line[3]), int(line[5])])
                    ip_address = line[-1].replace('[', '').replace(']', '')
                    as_number = get_AS_number(ip_address)
                else:
                    ip_address = line[1]
                    rttd = np.average([float(line[2][ : -3]), float(line[3][ : -3]), float(line[4][ : -3])])
                    as_number = get_AS_number(ip_address)
                source = prev_as_number
                if flag:
                    source = sources_label[source_i]
                flag = 0
                
            elif source_i == 1:
                try:
                    tmp = int(line[0])
                    ip_address = line[2].replace('(', '').replace(')', '')
                    rttd = float(line[3])
                    as_number = get_AS_number(ip_address)
                except:
                    continue
                source = prev_as_number
                if flag:
                    source = sources_label[source_i]
                    tmp_edge_list[-1][1] = destinations_label[i_dest]
                flag = 0
                
            elif source_i == 2:
                rttd = np.average([float(line[1]), float(line[3]), float(line[5])])
                ip_address = line[-1].replace('[', '').replace(']', '')
                as_number = get_AS_number(ip_address)
                source = prev_as_number
                if flag:
                    source = sources_label[source_i]
                    tmp_edge_list[-1][1] = destinations_label[i_dest]
                flag = 0
                
            elif source_i == 3:
                # if line[2] == 'ms':
                #     rttd = np.average([int(line[1]), int(line[3]), int(line[5])])
                #     ip_address = line[-1].replace('[', '').replace(']', '')
                #     as_number = get_AS_number(ip_address)
                # else:
                ip_address = line[2].replace('(', '').replace(')', '')
                rttd = np.average([float(line[-2]), float(line[-4])])
                as_number = get_AS_number(ip_address)
                source = prev_as_number
                
                if flag:
                    source = sources_label[source_i]
                    tmp_edge_list[-1][1] = destinations_label[i_dest]
                flag = 0
                
            else:
                rttd = float(line[-2])
                ip_address = line[-3].replace('(', '').replace(')', '')
                as_number = get_AS_number(ip_address)
                source = prev_as_number
                
                if flag:
                    source = sources_label[source_i]
                    tmp_edge_list[-1][1] = destinations_label[i_dest]
                flag = 0
                
            rttd = int(100 * rttd) / 100.0
            print("IP Address: ", ip_address)
            print("Round Trip Delay: ", rttd) 
            print("AS Number: ", as_number)
            print(f"\n")
            
            if i > 0:
                target = as_number
                edge = [source, prev_ip_address, target, ip_address, abs(rttd - rttd_prev)]
                tmp_edge_list.append(edge)
                tmp += 1
            prev_as_number, prev_ip_address, rttd_prev = as_number, ip_address, rttd
    tmp_edge_list[-1][1] = destinations_label[i_dest]

edge_list = tmp_edge_list
with open('output_v2.txt', 'w') as f:
    f.write(cap.stdout)

In [13]:
with open("./../Processed Data/IP_Address_list.txt", "w") as f:
    for ip_address in ip_address_list:
        f.write(ip_address + "\n")

with open("./../Processed Data/AS_Number_list.txt", "w") as f:
    for as_number in as_number_list:
        f.write(str(as_number) + "\n")

csv_filename = "./../Processed Data/Edge_list_ip_address.csv"
with open(csv_filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(fields) 
    csvwriter.writerows(edge_list)

In [14]:
as_number_list_new = []
for asn in as_number_list:
    try:
        as_number_list_new.append(int(asn))
    except:
        continue
np.unique(as_number_list_new)

array([  -100,      9,     14,    174,    786,   1299,   2381,   2497,
         2519,   2828,   2914,   3257,   3356,   3549,   4694,   4713,
         4755,   5089,   6453,   6461,   6830,   7922,   9371,   9498,
        11164,  11537,  13335,  14877,  20860,  20965,  22414,  35425,
        36236,  55824,  55836,  59193,  64049, 131965])